This notebook takes a list of Accessions Numbers in a csv file and saves the studies to the specified output path

In [ ]:
acc_csv = ""
output_dir = ""

my_ip = ''
my_port = 
my_ae = b''

pacs_ip = ''
pacs_port = 
pacs_ae = b''

In [ ]:
import os, csv, datetime
import pandas as pd

#import tqdm.notebook as tqdm

from pydicom import dcmread
from pydicom.dataset import Dataset

from pynetdicom import AE, evt
from pynetdicom.sop_class import StudyRootQueryRetrieveInformationModelFind
from pydicom import filewriter
from pydicom.uid import ImplicitVRLittleEndian, JPEGBaseline

from pynetdicom import AE, evt, StoragePresentationContexts
from pynetdicom.sop_class import StudyRootQueryRetrieveInformationModelMove, StudyRootQueryRetrieveInformationModelGet
from pynetdicom.sop_class import CTImageStorage, DigitalXRayImageProcessingStorage

In [ ]:
ae = AE(ae_title = my_ae)

# Add a requested presentation context
ae.add_requested_context(StudyRootQueryRetrieveInformationModelFind)

In [ ]:
def make_folder(folder):
    if not os.path.isdir(folder):
        os.mkdir(folder)

In [ ]:
def create_ds(acc):
    ds = Dataset()
    ds.QueryRetrieveLevel = 'STUDY'
    ds.AccessionNumber = acc
    ds.StudyInstanceUID = ''
    return ds

In [ ]:
def make_study_folder(acc):
    os.mkdir(os.path.join(output_dir, acc))

In [ ]:
def find_study(acc):
    outputFrame = pd.DataFrame()
    # Associate with peer AE at IP 127.0.0.1 and port 11112
    assoc = ae.associate(pacs_ip, pacs_port, ae_title = pacs_ae)
    if assoc.is_established:
        # Use the C-FIND service to send the identifier
        # A query_model value of 'P' means use the 'Patient Root Query Retrieve
        #     Information Model - Find' presentation context
        responses = assoc.send_c_find(ds, query_model='S')
        for (status, identifier) in responses:
            if status:
                # If the status is 'Pending' then identifier is the C-FIND response
                if status.Status in (0xFF00, 0xFF01):
                    outputFrame = outputFrame.append({'AccessionNumber':identifier.AccessionNumber,'StudyInstanceUID':identifier.StudyInstanceUID}, ignore_index=True)
            else:
                outputFrame = outputFrame.append({'AccessionNumber':ds.AccessionNumber,'StudyInstanceUID':'Error'}, ignore_index=True)



    else:
        print('Association rejected, aborted or never connected')
    # Release the association
    assoc.release()
    return outputFrame


In [ ]:
def handle_store(event):
#     print("handle_store called")
    """Handle a C-STORE request event."""
    ds = event.dataset
    context = event.context
    # Set the transfer syntax attributes of the dataset
    ds.is_little_endian = context.transfer_syntax.is_little_endian
    ds.is_implicit_VR = context.transfer_syntax.is_implicit_VR  
    # Save the dataset using the SOP Instance UID as the filename
    output = os.path.join(output_dir, ds.AccessionNumber, ds.SOPInstanceUID + ".dcm")
#     print(output)
    ds.save_as(output, write_like_original=False)

    # Return a 'Success' status
    return 0x0000

def data_recv(event):
#     print("data recieved")
#     print(event)
    return 0x0000

def established(event):
    print("association established")
    return 0x0000
def rejected(event):
    print("association rejected")
    return 0x0000
def accepted(event):
    print("association accepted")
    return 0x0000
def aborted(event):
    print("association aborted")
    return 0x0000

In [ ]:
def create_move_ds(AccessionNumber, StudyInstanceUID):
    ds = Dataset()
    ds.QueryRetrieveLevel = 'STUDY'
    ds.AccessionNumber = AccessionNumber
    ds.StudyInstanceUID = StudyInstanceUID
    return ds

In [ ]:
handlers = [(evt.EVT_ESTABLISHED, established),
        (evt.EVT_REJECTED, rejected),
        (evt.EVT_ACCEPTED, accepted),
        (evt.EVT_ABORTED, aborted),
        (evt.EVT_C_MOVE, handle_store),
        (evt.EVT_C_STORE, handle_store),
        (evt.EVT_DATA_RECV, data_recv)]

def start_move_ae():
    # Initialise the Application Entity
    ae = AE()

    # Add a requested presentation context
    ae.add_requested_context(StudyRootQueryRetrieveInformationModelMove)

    # Add the Storage SCP's supported presentation contexts
    ae.supported_contexts = StoragePresentationContexts

    # Start our Storage SCP in non-blocking mode, listening on port 11120
    ae.ae_title = my_ae
    
    return ae

def start_scp():
    scp = ae.start_server((my_ip, my_port), block=False, evt_handlers=handlers, ae_title=pacs_ae)
    return scp


In [ ]:
def move_study(ds):
    assoc = ae.associate(pacs_ip, pacs_port, ae_title=pacs_ae)
    if assoc.is_established:
    # Use the C-MOVE service to send the identifier
        responses = assoc.send_c_move(ds, my_ae, StudyRootQueryRetrieveInformationModelMove)
        for (status, identifier) in responses:
            if status:
#                 print('C-MOVE query status: 0x{0:04x}'.format(status.Status))
                # If the status is 'Pending' then `identifier` is the C-MOVE response
                if status.Status in (0xFF00, 0xFF01):
                    pass
#                     print(identifier)
            else:
                print('Connection timed out, was aborted or received invalid response')
    else:
        print('Association rejected, aborted or never connected')
    assoc.release()


In [ ]:
with open(acc_csv, 'r') as f:
    reader = csv.reader(f)
    acc_list = list(reader)
    
studies = pd.DataFrame()

for(acc) in acc_list:
    ds = create_ds(acc)
    studies = studies.append(find_study(ds), ignore_index=True)

In [ ]:
make_folder(output_dir)
# handlers = set_handlers()
ae = start_move_ae()
scp = start_scp()

In [ ]:
for index, row in studies.iterrows():
    make_study_folder(row["AccessionNumber"])
    ds = create_move_ds(row["AccessionNumber"], row["StudyInstanceUID"])
    move_study(ds)

In [ ]:
# Stop our Storage SCP
scp.shutdown()